In [1]:
!pip install torch

  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 22.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 16.9 MB/s eta 0:00:00a 0:00:01
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.5-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl (12 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


# Key-Value Attention

In [4]:
import torch


def key_value_attention(query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, k_v_cache: dict) -> torch.Tensor:

    if not k_v_cache:
        k_v_cache = {"key": [], "value": []}
    if k_v_cache["key"] is None:
        k_v_cache["key"] = key
        k_v_cache["value"] = value
    else:
        k_v_cache["key"] = torch.cat([k_v_cache["key"], key], dim=1)
        k_v_cache["value"] = torch.cat([k_v_cache["value"], value], dim=1)
    # Compute attention
    scale = 1/ torch.sqrt(torch.tensor(query.shape[-1])) 
    compatibility = torch.matmul(query, k_v_cache["key"].transpose(-2, -1)) # (batch_size, seq_len, seq_len)
    attention = torch.softmax(compatibility * scale, dim=-1) # (batch_size, seq_len, seq_len)
    weighted_attention = torch.matmul(attention, k_v_cache["value"]) # (batch_size, seq_len, d_value)

    return weighted_attention, k_v_cache




In [5]:
# Test
query = torch.randn(1, 1, 10)
key = torch.randn(1, 1, 10)
value = torch.randn(1, 1, 10)

print(key_value_attention(query, key, value))

tensor([[[ 0.0574,  1.3514,  0.4958,  0.0324,  0.1390,  1.5929,  0.3412,
           1.3585, -0.2910, -0.5900]]])
